In [21]:
import math
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import jax
import jax.numpy as jnp
import jax.random as jr
import jax.nn as jnn
from jax import ops
import matplotlib.pyplot as plt
import seaborn as sns
import time as time

from sklearn.preprocessing import LabelEncoder
# from tslearn.datasets import UCR_UEA_datasets
# from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pywt

from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
# import os
# import sys
from joblib import dump, load

import utility
#sys.path.append('/kaggle/usr/lib/utility')
import sigkerax
from sigkerax.sigkernel import SigKernel

# Submission notebook: no training
Loading models and applying to a generic test dataframe

In [19]:
PATH_TRAIN = '/kaggle/input/hms-harmful-brain-activity-classification/train.csv'
EEG_PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
FEATS = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
df = pd.read_csv(PATH_TRAIN)
TARGETS = df.columns[-6:]
df_unique = df.drop_duplicates(subset=['eeg_id', 'seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote'], keep='first')
df_unique[TARGETS] = df_unique[TARGETS].astype('float64')
y = df_unique[TARGETS].agg('sum', axis=1)
df_unique.loc[:,'sum_votes']=y
# strong samples
df_unique = df_unique.loc[df_unique['sum_votes']>10]
#unanimous samples
# df_unique['max_category_votes'] = df_unique[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].max(axis=1)
# df_unique = df_unique[df_unique['max_category_votes'] == df_unique['sum_votes']]
# df_unique.drop(columns=['max_category_votes'], inplace=True)
#df_unique.loc[:, TARGETS] = v.astype('float64')
len(df_unique)

/tmp/ipykernel_33/774425918.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique[TARGETS] = df_unique[TARGETS].astype('float64')
/tmp/ipykernel_33/774425918.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.loc[:,'sum_votes']=y


6233

In [3]:
PATH_TEST = '/kaggle/input/hms-harmful-brain-activity-classification/test.csv'
EEG_PATH_TEST = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'
FEATS = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
df_test = pd.read_csv(PATH_TEST)

In [17]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)
def denoise(x, wavelet='haar', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    ret=pywt.waverec(coeff, wavelet, mode='per')

    return ret
def montage(eeg_data, electrodes=FEATS):
    """
    Apply bipolar montaging to EEG data.
    
    Parameters:
    - eeg_data: NumPy array with shape (timepoints, channels), where channels correspond to electrodes.
    - electrodes: List of electrode names in the same order as the eeg_data channels.
    
    Returns:
    - montaged_data: NumPy array after applying bipolar montaging.
    """
    # Define pairs for bipolar montage based on electrode positions
    # This is a simplified example; pairs should be chosen based on electrode locations and analysis needs
    pairs = [('Fp1', 'T3'), ('T3', 'O1'), ('Fp1', 'C3'), ('C3', 'O1'),
             ('Fp2', 'C4'), ('C4', 'O2'), ('Fp2', 'T4'), ('T4', 'O2')]
    
    montaged_data = []
    for pair in pairs:
        idx1 = electrodes.index(pair[0])
        idx2 = electrodes.index(pair[1])
        # Compute the difference between the pairs
        montaged_data.append(eeg_data[:, idx1] - eeg_data[:, idx2])
    
    # Stack the montaged data to form a new array
    montaged_data = np.stack(montaged_data, axis=-1)
    
    return montaged_data
def ft(eeg_data):
    # Apply Fast Fourier Transform (FFT) to each channel
    fft_results = np.fft.rfft(eeg_data, axis=0)
    
    # Compute magnitude spectrum (amplitude)
    magnitude_spectrum = np.abs(fft_results)
    
    # Optionally, compute the power spectrum or log power spectrum for analysis
    power_spectrum = np.square(magnitude_spectrum)
    log_power_spectrum = np.log(power_spectrum + 1e-10)  # Adding epsilon to avoid log(0)
    
    return log_power_spectrum

def fft_filter(signal, fs):
    """
    Filters the EEG signal to retain important brain wave frequencies
    and reduces the size by a factor of 8.
    
    Parameters:
    - signal: The time-domain EEG signal (1D numpy array).
    - fs: Sampling frequency of the signal.
    
    Returns:
    - reduced_signal: The time-domain signal after filtering and size reduction.
    """
    # Perform FFT
    fft_result = np.fft.fft(signal)
    freq = np.fft.fftfreq(len(signal), d=1/fs)
    
    # Define brain wave frequency ranges
    delta = (0.5, 4)
    theta = (4, 8)
    alpha = (8, 12)
    beta = (12, 30)
    
    # Create a mask to retain frequencies within the defined brain wave ranges
    mask = ((np.abs(freq) >= delta[0]) & (np.abs(freq) <= delta[1])) | \
           ((np.abs(freq) >= theta[0]) & (np.abs(freq) <= theta[1])) | \
           ((np.abs(freq) >= alpha[0]) & (np.abs(freq) <= alpha[1])) | \
           ((np.abs(freq) >= beta[0]) & (np.abs(freq) <= beta[1]))
    
    # Apply mask and zero out other frequencies
    fft_result_filtered = np.zeros_like(fft_result)
    fft_result_filtered[mask] = fft_result[mask]
    
    # Optionally, reduce the size of the filtered FFT result by a factor of 8
    # This simplistic approach keeps every 8th frequency component
    reduction_factor = 2
    reduced_fft_result_filtered = fft_result_filtered[::reduction_factor]
    
    # Perform Inverse FFT to get the filtered, reduced signal back in time domain
    reduced_signal = np.fft.ifft(reduced_fft_result_filtered).real
    
    return reduced_signal[::4]

def load_data_batch(batch_size, batch_num, df= df_unique):
    test_mode = df.equals(df_test)
    EEG_PATH = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/' if test_mode else '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
    start_index = batch_num * batch_size
    end_index = start_index + batch_size
    batch_df = df.iloc[start_index:end_index]
    data = []

    for row in batch_df.itertuples():
        eeg_id = getattr(row, 'eeg_id')
        offset = int(getattr(row, 'eeg_label_offset_seconds') * 200) if not test_mode else 0
        eeg = pq.read_table(f"{EEG_PATH}{eeg_id}.parquet").to_pandas().iloc[offset + 4000:offset + 6000]
        eeg = eeg.loc[:, eeg.columns.isin(FEATS)].ffill().bfill().values
        #eeg = montage(eeg)
        #eeg = denoise(eeg)
        eeg = fft_filter(eeg,200)
        #eeg = montage(eeg)
        
        max_val = np.abs(eeg).max(axis=0, keepdims=True)
        max_val[max_val == 0] = 1  # Replace 0s with 1s to avoid division by zero
        eeg /= max_val
        data.append(eeg)

    # Convert the list of numpy arrays to a single JAX array
    return jnp.array(data)

In [5]:
def construct_kernel_matrix(df1, df2, sigma, signature_kernel, num_batches = 10):
    n = df1.shape[0]
    m = df2.shape[0]
    global_kernel_matrix = jnp.zeros((n,m))
    
    batch_size = math.ceil(n / num_batches)
    # initialize signature PDE kernel
#     signature_kernel = SigKernel(refinement_factor=1, static_kernel_kind="rbf", scales = jnp.array([sigma]), add_time=True)
    for i in range(num_batches):
        # load batch i
        t1 = time.time()
        X_batch = load_data_batch(batch_size, i, df=df1)
        #X_batch = X_batch[:,::8,:]
        t2 = time.time()
        print(f"load one batch time: {t2-t1}")
        for j in tqdm(range(i, num_batches)):  # Use range(i, num_batches) to avoid redundant calculations
            # Load batch j
            Y_batch = load_data_batch(batch_size, j, df=df2)
            # slice and scale
            #Y_batch = Y_batch[:,::8,:]
#             X_batch *= 0.001
#             Y_batch *= 0.001
            t1 = time.time()
            # Compute the kernel matrix between these two batches
            local_kernel_matrix = signature_kernel.kernel_matrix(X_batch, Y_batch)[...,0]
            t2 = time.time()
            print(f"local kernel time: {t2-t1}")
            #Insert the local kernel matrix into the global matrix
            start_i, end_i = i * batch_size, (i + 1) * batch_size
            start_j, end_j = j * batch_size, (j + 1) * batch_size
            t1 = time.time()
            global_kernel_matrix = global_kernel_matrix.at[start_i:end_i, start_j:end_j].set(local_kernel_matrix)
            t2 = time.time()
            #print(f"assign to global kernel time: {t2-t1}")
            # For the non-diagonal blocks, where i != j, fill the symmetric position in the matrix
            if i != j:
                global_kernel_matrix = global_kernel_matrix.at[start_j:end_j, start_i:end_i].set(local_kernel_matrix.T)
    return jnp.array(global_kernel_matrix)

def test_matrix(df1, df2, sigma, signature_kernel, batch_size=10):
    n = df1.shape[0]
    m = df2.shape[0]
    global_kernel_matrix = jnp.zeros((n, m))
    
    # Calculate the number of batches for each dataframe
    num_batches_df1 = math.ceil(n / batch_size)
    num_batches_df2 = math.ceil(m / batch_size)
    
    for i in tqdm(range(num_batches_df1), desc="Computing kernel matrix", leave=False):
        # Dynamically compute start and end indices for df1 batches
        start_i = i * batch_size
        end_i = min(start_i + batch_size, n)
        X_batch = load_data_batch(end_i - start_i, i, df=df1)

        for j in range(num_batches_df2):
            # Dynamically compute start and end indices for df2 batches
            start_j = j * batch_size
            end_j = min(start_j + batch_size, m)
            Y_batch = load_data_batch(end_j - start_j, j, df=df2)
            
            # Compute the kernel matrix between X_batch and Y_batch
            local_kernel_matrix = signature_kernel.kernel_matrix(X_batch, Y_batch)[..., 0]
            
            # Insert the local kernel matrix into the appropriate slice of the global matrix
            global_kernel_matrix = global_kernel_matrix.at[start_i:end_i, start_j:end_j].set(local_kernel_matrix)
    del(X_batch)
    del(Y_batch)
    return global_kernel_matrix

In [6]:
def kld(p, q):
    """
    Calculate the Kullback-Leibler Divergence between two probability distributions.
    
    Parameters:
    - p: The true probability distribution (as a NumPy array).
    - q: The predicted probability distribution (as a NumPy array).
    
    Returns:
    - The KL Divergence value.
    """
    # Ensure the distributions sum to 1 (if they don't already)
    p_normalized = p / np.sum(p)
    q_normalized = q / np.sum(q)
    
    # Only consider non-zero elements
    nonzero_indices = (p_normalized > 0) & (q_normalized > 0)
    
    return np.sum(p_normalized[nonzero_indices] * np.log(p_normalized[nonzero_indices] / q_normalized[nonzero_indices]))


In [20]:
df_unique = df_unique.iloc[:4000]
model_path = '/kaggle/input/final-fft-model/kernel_ridge_model_fft_filter_final_model.joblib'
krr = load(model_path)
sigma = 0.01

signature_kernel = SigKernel(refinement_factor=2, static_kernel_kind="linear", scales = jnp.array([sigma]), add_time=True)

K_test = test_matrix(df_test, df_unique, sigma, signature_kernel, 500)
test_predictions = krr.predict(K_test)


In [22]:
scaled_predictions = (jnn.relu(test_predictions).T/jnn.relu(test_predictions).sum(axis = 1)).T

In [9]:
submission = pd.DataFrame({'eeg_id': df_test.eeg_id.values})
labels=['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']
submission[labels] = scaled_predictions
submission.to_csv("submission.csv",index=None)
display(submission.head())

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.051843,0.187102,0.121749,0.083182,0.087862,0.468262
